In [1]:
import pandas as pd
import sklearn 
import scipy
from scipy.stats import shapiro
from sklearn import linear_model as lm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge , ElasticNet, LogisticRegression, lars_path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold, train_test_split, cross_val_score, StratifiedKFold, LabelKFold, ShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
import matplotlib.pyplot as plt 
% matplotlib inline
import numpy as np
import math 
import seaborn as sns
import statsmodels.api as sm
import statsmodels
from statsmodels import graphics
from statsmodels.graphics.gofplots import qqplot
import datetime
from datetime import timedelta, time

/Users/giancarlokerg/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_day = pd.read_csv("day.csv")
df_hour = pd.read_csv("hour.csv")

In [3]:
df_hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
df_hour.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [5]:
dummies_season_hour = pd.get_dummies(df_hour['season']).rename(columns={1: "Spring", 2: "Summer", 3: "Fall", 4: "Winter"})
dummies_yr_hour = pd.get_dummies(df_hour['yr']).rename(columns={0: "year 2011", 1: "year 2012"})
dummies_mnth_hour = pd.get_dummies(df_hour['mnth']).rename(columns={1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"})
dummies_holiday_hour = pd.get_dummies(df_hour['holiday']).rename(columns={0: "holiday", 1: "non holiday"})
dummies_weekday_hour = pd.get_dummies(df_hour['weekday']).rename(columns={0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"})
dummies_workingday_hour = pd.get_dummies(df_hour['workingday']).rename(columns={0: "workingday", 1: "non workingday"})
dummies_weathersit_hour = pd.get_dummies(df_hour['weathersit']).rename(columns={1: "weathersit 1", 2: "weathersit 2", 3: "weathersit 3", 4: "weathersit 4"})
dummies_hr_hour = pd.get_dummies(df_hour['hr']).rename(columns = {0: "hr 0", 1: "hr 1", 2: "hr 2", 3: "hr 3", 4: "hr 4", 5: "hr 5", 6: "hr 6", 7: "hr 7",8: "hr 8", 9: "hr 9",10: "hr 10", 11: "hr 11", 12: "hr 12", 13: "hr 13", 14: "hr 14", 15: "hr 15", 16: "hr 16", 17: "hr 17", 18: "hr 18", 19: "hr 19", 20: "hr 20", 21: "hr 21", 22: "hr 22", 23: "hr 23"})
dummy_features_hour = pd.concat([dummies_season_hour,dummies_yr_hour,dummies_mnth_hour, dummies_holiday_hour,dummies_weekday_hour,dummies_workingday_hour,dummies_weathersit_hour, dummies_hr_hour],axis=1)

In [6]:
dummies_season = pd.get_dummies(df_day['season']).rename(columns={1: "Spring", 2: "Summer", 3: "Fall", 4: "Winter"})
dummies_yr = pd.get_dummies(df_day['yr']).rename(columns={0: "year 2011", 1: "year 2012"})
dummies_mnth = pd.get_dummies(df_day['mnth']).rename(columns={1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"})
dummies_holiday = pd.get_dummies(df_day['holiday']).rename(columns={0: "holiday", 1: "non holiday"})
dummies_weekday = pd.get_dummies(df_day['weekday']).rename(columns={0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"})
dummies_workingday = pd.get_dummies(df_day['workingday']).rename(columns={0: "workingday", 1: "non workingday"})
dummies_weathersit = pd.get_dummies(df_day['weathersit']).rename(columns={1: "weathersit 1", 2: "weathersit 2", 3: "weathersit 3", 4: "weathersit 4"})
dummy_features = pd.concat([dummies_season,dummies_yr,dummies_mnth, dummies_holiday,dummies_weekday,dummies_workingday,dummies_weathersit],axis=1)

In [7]:
cont_features= df_day[['temp', 'atemp', 'hum', 'windspeed']]

In [8]:
cont_features_hour= df_hour[['temp', 'atemp', 'hum', 'windspeed']]

In [9]:
X = pd.concat([cont_features, dummy_features],axis = 1)
y_day = df_day['cnt']
y_day_cas = df_day['casual']
y_day_reg = df_day['registered']
X_work, X_eval, y_work, y_eval = train_test_split(X,y_day , test_size = 0.1, random_state=3)
X_work, X_eval, y_work_cas, y_eval_cas = train_test_split(X,y_day , test_size = 0.1, random_state=3)
X_work, X_eval, y_work_reg, y_eval_reg = train_test_split(X,y_day , test_size = 0.1, random_state=3)

In [10]:
X_hour = pd.concat([cont_features_hour, dummy_features_hour],axis = 1)
y_hour = df_hour['cnt']
y_hour_cas = df_hour['casual']
y_hour_reg = df_hour['registered']

let us now define the map **kappa** that sends indices from the daily data set to corresponding indices in the hourly data set so that the dates of both index sets coincide.

In [11]:
def kappa(index_list):
    date_index_list = []
    for date in list(df_day['dteday'][index_list]):
        date_index_list = date_index_list + list(df_hour[df_hour['dteday'] == date].index)
    return date_index_list

In [12]:
X_work_hour = X_hour.loc[kappa(X_work.index)] # we want the same hours to be kept out that correspond to days kept out initially    

In [13]:
X_work_hour.head()

,temp,atemp,hum,windspeed,Spring,Summer,Fall,Winter,year 2011,year 2012,...,hr 14,hr 15,hr 16,hr 17,hr 18,hr 19,hr 20,hr 21,hr 22,hr 23
11131,0.40,0.4091,0.58,0.1343,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11132,0.40,0.4091,0.58,0.1343,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11133,0.38,0.3939,0.66,0.1343,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11134,0.36,0.3636,0.66,0.0896,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11135,0.36,0.3636,0.71,0.1045,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let `n_estimators_day`, `max_depth_day`, `learning_rate_day ` be the best parameters found for the daily prediction (without the augmented dataset)

In [ ]:
n_estimators_day= 
max_depth_day = 
learning_rate_day = 
folds = 3
best_score = 2000

for n_estimators in [1000,1500,2000,2500]:
    for learning_rate in [0.001, 0.01, 0.1 , 1]:
        for max_depth in [1,2,3,4]:
            score = 0
            folds = 3
            for i, (train_idx, test_idx) in enumerate(KFold(len(X_work),folds, shuffle=True)): 
                train = X_work.index[train_idx]
                test = X_work.index[test_idx]
                GBR = GradientBoostingRegressor(n_estimators = n_estimators_day , max_depth = max_depth_day, learning_rate = learning_rate_day, criterion='mae').fit(X_work.loc[train], y_work.loc[train])
                GBR_hour = GradientBoostingRegressor(n_estimators = n_estimators , max_depth = max_depth, learning_rate = learning_rate, criterion='mae').fit(X_work_hour.loc[kappa(train)], y_hour.loc[kappa(train)]-GBR.predict(X_work.loc[train])/24)
                score = score + mean_absolute_error(y_hour.loc[kappa(test)], GBR_hour.predict(X_work_hour.loc[kappa(test)]))
                MAE = score/3 
            if MAE < best_score: 
                    best_score = MAE
                    best_parameters = {'learning_rate': learning_rate, 'n_estimtors': n_estimators, 'max_depth': max_depth}

print("best score:", best_score)
print("best parameters: ", best_parameters)

Let us take back the data set from previous notebook taken from http://aa.usno.navy.mil/data/docs/RS_OneYear.php  

In [14]:
df_2011 = pd.read_table("sunrise_sunset_2011.txt", delim_whitespace=True, dtype = 'str') 
df_2012 = pd.read_table("sunrise_sunset_2012.txt", delim_whitespace=True, dtype = 'str')

In [15]:
df_2012.head()

,Day,Rise_1,Set_1,Rise_2,Set_2,Rise_3,Set_3,Rise_4,Set_4,Rise_5,...,Rise_8,Set_8,Rise_9,Set_9,Rise_10,Set_10,Rise_11,Set_11,Rise_12,Set_12
0,01,0727,1657,0715,1729,0640,1801,0552,1832,0510,...,0510,1919,0537,1838,0604,1750,0636,1707,0708,1646
1,02,0727,1657,0714,1730,0638,1802,0551,1833,0509,...,0510,1918,0538,1836,0605,1749,0637,1706,0709,1646
2,03,0727,1658,0713,1731,0637,1803,0549,1834,0507,...,0511,1917,0539,1835,0606,1747,0638,1705,0710,1646
3,04,0727,1659,0712,1732,0635,1805,0547,1835,0506,...,0512,1915,0540,1833,0607,1745,0639,1704,0711,1646
4,05,0727,1700,0711,1734,0634,1806,0546,1836,0505,...,0513,1914,0541,1831,0608,1744,0640,1703,0712,1646


In [19]:
start_date = datetime.date(2011, 1, 1)
end_date   = datetime.date(2013, 1, 1)

dates_2011_2012 = [ start_date + datetime.timedelta(n) for n in range(int ((end_date - start_date).days))]
df = pd.DataFrame(dates_2011_2012, columns= ['date'])

def get_hour(date):
    return date.hour
def get_day(date):
    return date.day
def get_month(date):
    return date.month
def get_year(date):
    return date.year
def get_isoformat(date):
    return date.isoformat()

df['day'] = df['date'].apply(get_day)
df['mnth'] = df['date'].apply(get_month)
df['year'] = df['date'].apply(get_year)
df['dteday'] = df['date'].apply(get_isoformat)

for j in range(12):
    index_list_2011 = df[(df['mnth']== (j+1))&(df['year']==2011)].index.get_values()
    index_list_2012 = df[(df['mnth']== (j+1))&(df['year']==2012)].index.get_values()
    
    for k,i in enumerate(index_list_2011):
        df.loc[i,'Rise'] = df_2011['Rise_'+str(j+1)][k] 
        df.loc[i,'Set'] = df_2011['Set_'+str(j+1)][k]
        
    for k,i in enumerate(index_list_2012):
        df.loc[i,'Rise'] = df_2012['Rise_'+str(j+1)][k] 
        df.loc[i,'Set'] = df_2012['Set_'+str(j+1)][k]

def str_to_datetime(time):
    return datetime.time(hour = int(time[0:2]), minute = int(time[2:5]))
def convert_to_minutes(time):
    t= datetime.time(hour = int(time[0:2]), minute = int(time[2:5]))
    return t.hour*60 + t.minute

df['Rise datetime']=df['Rise'].apply(str_to_datetime)
df['Set datetime']=df['Set'].apply(str_to_datetime)
df['daylight exposure'] = df['Set'].apply(convert_to_minutes)-df['Rise'].apply(convert_to_minutes)

df_day['daylight exposure'] = df['daylight exposure']

In [23]:
start_date_hour = datetime.datetime(2011, 1, 1, 0)
end_date_hour   = datetime.datetime(2013, 1, 1, 0)

m = (end_date_hour.year-start_date_hour.year)*365*24+(end_date_hour.month - start_date_hour.month)*30*24 + (end_date_hour.day - start_date_hour.day)*24+(end_date_hour.hour - start_date_hour.hour)
dates_2011_2012_hour = [ start_date_hour + datetime.timedelta(hours = n) for n in range(m)]
df_hourly = pd.DataFrame(dates_2011_2012_hour, columns= ['date'])

df_hourly['hour'] = df_hourly['date'].apply(get_hour)
df_hourly['day'] = df_hourly['date'].apply(get_day)
df_hourly['mnth'] = df_hourly['date'].apply(get_month)
df_hourly['year'] = df_hourly['date'].apply(get_year)
df_hourly['dteday'] = df_hourly['date'].apply(get_isoformat)

def brightness(date_time):
    r = df['Rise datetime'][int(df[df['date'] == date_time.date()].index.get_values())]
    s = df['Set datetime'][int(df[df['date'] == date_time.date()].index.get_values())]
    result = 0
    if (date_time.hour == r.hour):
        result = 1-(r.minute /60)
    elif (date_time.hour == s.hour):
        result = s.minute /60
    elif (r.hour< date_time.hour <s.hour):
        result = 1
    else:
        result = 0
    return result

df_hourly['brightness'] = df_hourly['date'].apply(brightness)
df_hour['brightness'] = df_hourly['brightness']
X_plus = pd.concat([X, df_day['daylight exposure']],axis=1)
X_work_plus = X_plus.loc[X_work.index]

In [24]:
X_plus.head()

,temp,atemp,hum,windspeed,Spring,Summer,Fall,Winter,year 2011,year 2012,...,Wednesday,Thursday,Friday,Saturday,workingday,non workingday,weathersit 1,weathersit 2,weathersit 3,daylight exposure
0,0.344167,0.363625,0.805833,0.160446,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,570
1,0.363478,0.353739,0.696087,0.248539,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,571
2,0.196364,0.189405,0.437273,0.248309,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,571
3,0.200000,0.212122,0.590435,0.160296,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,572
4,0.226957,0.229270,0.436957,0.186900,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,573


In [25]:
X_hour_plus = pd.concat([X_hour, df_hour['brightness']],axis=1)

In [26]:
X_hour_plus.head()

,temp,atemp,hum,windspeed,Spring,Summer,Fall,Winter,year 2011,year 2012,...,hr 15,hr 16,hr 17,hr 18,hr 19,hr 20,hr 21,hr 22,hr 23,brightness
0,0.24,0.2879,0.81,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_work_hour_plus = X_hour_plus.loc[kappa(X_work.index)]

Let `n_estimators_day_plus`, `max_depth_day_plus`, `learning_rate_day_plus ` be the best parameters found for the daily prediction (without the augmented dataset)

In [ ]:
n_estimators_day_plus= 
max_depth_day_plus = 
learning_rate_day_plus = 
folds = 3
best_score = 2000

for n_estimators in [1000,1500,2000,2500]:
    for learning_rate in [0.001, 0.01, 0.1 , 1]:
        for max_depth in [1,2,3,4]:
            score = 0
            folds = 3
            for i, (train_idx, test_idx) in enumerate(KFold(len(X_work),folds, shuffle=True)): 
                train = X_work.index[train_idx]
                test = X_work.index[test_idx]
                GBR = GradientBoostingRegressor(n_estimators = n_estimators_day_plus , max_depth = max_depth_day_plus, learning_rate = learning_rate_day_plus, criterion='mae').fit(X_work_plus.loc[train], y_work.loc[train])
                GBR_hour = GradientBoostingRegressor(n_estimators = n_estimators , max_depth = max_depth, learning_rate = learning_rate, criterion='mae').fit(X_work_hour_plus.loc[kappa(train)], y_hour.loc[kappa(train)]-GBR.predict(X_work_plus.loc[train])/24)
                score = score + mean_absolute_error(y_hour.loc[kappa(test)], GBR_hour.predict(X_work_hour.loc[kappa(test)]))
                MAE = score/3 
            if MAE < best_score: 
                    best_score = MAE
                    best_parameters = {'learning_rate': learning_rate, 'n_estimtors': n_estimators, 'max_depth': max_depth}

print("best score:", best_score)
print("best parameters: ", best_parameters)

for evaluation, just take the best found parameters, and evaluate on the held out 0.1 sized test set 